In [105]:
############################################################################################################
## 1. 패키지 import
############################################################################################################
import pandas as pd
import numpy as np
import sqlite3
import sys
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pickle import dump, load
import warnings
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

# User Package
from module.logger import log_message
import module.sqlTransaction as sqlT
import importlib
importlib.reload(sqlT)

############################################################################################################
## 2. 초기설정
############################################################################################################
# 경로설정
dir_work   = f'c:/Users/user/OneDrive - 파인트리파트너스(주)/movie'
dir_func   = f'{dir_work}/src/module'
dir_data   = f'{dir_work}/data'
dir_mdl    = f'{dir_work}/data/trn'

# DB연결
conn = sqlite3.connect(f"{dir_data}/pine_movie.db", isolation_level=None)
cur = conn.cursor()

# 함수 호출
# exec(open(f"{dir_func}/sqlTransaction.py"      , encoding= 'utf-8').read() )
# exec(open(f"{dir_func}/logger.py"              , encoding= 'utf-8').read() )
exec(open(f"{dir_func}/func_mdl_iv_woe.py"              , encoding= 'utf-8').read() )

############################################################################################################
## 3. 파라미터 추출
############################################################################################################
# 입력 파라미터 추출
try :
    bas_ym = sys.argv[1]
    if not bas_ym.isdigit() : raise
except:
    bas_ym = '201208'
    
list_bas_ym   = [datetime.strftime(datetime.strptime(bas_ym, '%Y%m') - relativedelta(months = (i) ), '%Y%m') for i in range(3)]
bas_ym_bfr_12m = datetime.strftime(datetime.strptime(bas_ym, '%Y%m') - relativedelta(months = 12 ), '%Y%m')

# 인기영화 정보 생성

In [21]:
# 행동별 가중치 계산 
df_behav_cnt = pd.read_sql(f"""select 행동번호, count(기준년월) 건수 from movie_fact group by 행동번호""", conn) 
df_behav_cnt['전체건수'] = df_behav_cnt['건수'].sum()
df_behav_cnt['비율'] = df_behav_cnt['건수'] / df_behav_cnt['전체건수']
df_behav_cnt['가중치'] = 1 / df_behav_cnt['비율'] 
df_behav_cnt

,행동번호,건수,전체건수,비율,가중치
0,1,296915,3180156,0.093365,10.710661
1,2,455497,3180156,0.143231,6.981728
2,4,503518,3180156,0.158331,6.315874
3,5,1477320,3180156,0.464543,2.152652
4,11,446906,3180156,0.140530,7.115939


In [70]:
# 최근 1년 영화별 행동건수 정보 추출
df_movie_base = pd.read_sql(f"""
  select
          a01.영화번호
        , a03.영화명
        , a03.개봉년
        , a02.장르번호
        , count(case when a01.행동번호 = 5   then 1 else 0 end)                        as 탐색건수
        , count(case when a01.행동번호 = 11  then 1 else 0 end)                        as 구매건수
        , count(case when a01.행동번호 = 4   then 1 else 0 end)                        as 시작건수
        , count(case when a01.행동번호 = 2   then 1 else 0 end)                        as 완료건수
        , count(case when a01.행동번호 = 1   then 1 else 0 end)                        as 평점건수
  from MOVIE_FACT a01
  
  ----------------------------------------------------------------
  -- 장르 매핑
  ----------------------------------------------------------------
  left join (
      select
             영화번호
           , 장르번호
      from movie_genre
      --------------------------------------------
      -- 중복 장르 번호 제거
      --------------------------------------------
      except
      select
            b01.영화번호
          , b02.장르번호
      from MOVIE b01
  
      left join MOVIE_GENRE b02 -- 영화 장르 매핑
      on b01.영화번호 = b02.영화번호
  
      left join GENRE b03 -- 장르명 매핑
      on b02.장르번호 = b03.장르번호
  
      group by
            b01.영화번호
          , b03.장르명
      having count(1) > 1  -- 중복 존재 장르번호 추출
  ) a02
  on a01.영화번호 = a02.영화번호
  
  
  ----------------------------------------------------------------
  -- 영화정보 매핑
  ----------------------------------------------------------------
  left join movie a03
  on a01.영화번호 = a03.영화번호
  
  where a01.기준년월 between {bas_ym_bfr_12m} and {bas_ym}
  
  group by 
        a01.영화번호
      , a02.장르번호
"""
, conn)

In [102]:
# 인기점수 산출
df_movie_base['인기점수_원천'] = (
      df_movie_base['탐색건수'] * (df_behav_cnt[df_behav_cnt['행동번호'] == 5] ['가중치'].values[0])
    + df_movie_base['구매건수'] * (df_behav_cnt[df_behav_cnt['행동번호'] == 11]['가중치'].values[0])
    + df_movie_base['시작건수'] * (df_behav_cnt[df_behav_cnt['행동번호'] == 4 ]['가중치'].values[0])
    + df_movie_base['완료건수'] * (df_behav_cnt[df_behav_cnt['행동번호'] == 2 ]['가중치'].values[0])
    + df_movie_base['평점건수'] * (df_behav_cnt[df_behav_cnt['행동번호'] == 1 ]['가중치'].values[0])
)

# 인기점수 정규화
scaler = MinMaxScaler()
df_scale_target = df_movie_base['인기점수_원천'].to_frame()
scaler.fit(df_scale_target)
df_scaled = pd.DataFrame(scaler.transform(df_scale_target), columns=['인기도배점_정규화'])
df_scaled['인기점수'] = round(df_scaled * 100, 2)
df_scaled = pd.concat([df_movie_base, df_scaled], axis = 1)

# 최신영화 가점
df_scaled['인기점수'] = df_scaled.apply(lambda x: x['인기점수'] + 10 if (x['개봉년'] >= 2007)  else x['인기점수'], axis = 1)
df_scaled = df_scaled.sort_values(by='인기점수', ascending=False)
df_scaled = df_scaled[['영화번호','영화명','인기점수']].drop_duplicates(ignore_index=True)

# 라벨링 컬럼 추가
df_scaled['기준년월'] = bas_ym
df_scaled['장르']     = '공포'

In [117]:
# 인기점수 결과 테이블 DDL
cur.execute(f"""DROP TABLE MOVIE_RANK""")
cur.execute(f"""
    CREATE TABLE IF NOT EXISTS MOVIE_RANK  (
          기준년월
        , 영화번호
        , 장르
        , 영화명
        , 인기점수
        , PRIMARY KEY(기준년월, 영화번호)
    )
    """
)

# 인기점수 결과 적재
cur.executemany(
      "INSERT INTO MOVIE_RANK VALUES(?,?,?,?,?)"
    , list(df_scaled[['기준년월', '영화번호', '장르', '영화명', '인기점수']].itertuples(index=False, name=None))
)

# 장르 추천 결과와 결합

In [177]:
df_rcm_rslt = pd.read_sql(
f""" 
    SELECT
          기준년월
        , 회원번호
        , 장르
        , 영화번호
        , 영화명
        , 추천순위
    FROM (
        SELECT 
              T1.기준년월
            , T1.회원번호
            , T1.장르
            , T2.영화번호
            , T2.영화명
            , ROW_NUMBER() OVER (PARTITION BY T1.회원번호 ORDER BY T2.순위 )                           AS 추천순위

        FROM GENRE_RCM_RSLT T1
        
        --------------------------------------------
        -- 인기영화 순위 결합
        --------------------------------------------
        LEFT JOIN (
            SELECT
                기준년월
                , 영화번호
                , 영화명
                , rowid                    AS 순위
            FROM MOVIE_RANK
            WHERE 
                장르 = '공포'    
            ORDER BY 인기점수
        ) T2
        ON cast(T1.기준년월 as text) = T2.기준년월
        AND 순위 <= 100
        
        --------------------------------------------
        -- 기시청영화 제외
        --------------------------------------------
        LEFT JOIN (
            SELECT 
                DISTINCT 
                회원번호
                , 영화번호
                , 1                             AS 기시청여부
            FROM MOVIE_FACT
            WHERE 
                기준년월 < {bas_ym}
            AND 행동번호 != 5
        ) T3
        ON 
            T1.회원번호 = T3.회원번호
        AND T2.영화번호 = T3.영화번호
        
        WHERE 
            T1.장르     = '공포'   -- 공포 장르
        AND T1.추천여부 = 1        -- 추천 대상
        AND T3.기시청여부 IS NULL  -- 기시청영화는 비추천
    )
    WHERE 추천순위 <= 10
"""
, conn)
df_rcm_rslt

,기준년월,회원번호,장르,영화번호,영화명,추천순위
0,201208,1000693,공포,10193,Toy Story 3,1
1,201208,1000693,공포,22881,The Blind Side,2
2,201208,1000693,공포,6977,No Country for Old Men,3
3,201208,1000693,공포,18785,The Hangover,4
4,201208,1000693,공포,22954,Invictus,5
...,...,...,...,...,...,...
9920,201208,1446850,공포,769,Goodfellas,6
9921,201208,1446850,공포,6977,No Country for Old Men,7
9922,201208,1446850,공포,5915,Into the Wild,8
9923,201208,1446850,공포,8321,In Bruges,9


In [181]:
# 최종 추천 영화 테이블 DDL
# cur.execute(f"""DROP TABLE MOVIE_RCM_RSLT""")
# cur.execute(f"""
#     CREATE TABLE MOVIE_RCM_RSLT  (
#           기준년월
#         , 회원번호
#         , 영화번호
#         , 장르
#         , 영화명
#         , 추천순위
#     )
#     """
# )

# 인기점수 결과 적재
cur.executemany(
      "INSERT INTO MOVIE_RCM_RSLT VALUES(?,?,?,?,?,?)"
    , list(df_rcm_rslt[['기준년월', '회원번호', '영화번호', '장르', '영화명', '추천순위']].itertuples(index=False, name=None))
)

In [183]:
pd.read_sql(f" select * from movie_rcm_rslt ", conn)

,기준년월,회원번호,영화번호,장르,영화명,추천순위
0,201208,1000693,10193,공포,Toy Story 3,1
1,201208,1000693,22881,공포,The Blind Side,2
2,201208,1000693,6977,공포,No Country for Old Men,3
3,201208,1000693,18785,공포,The Hangover,4
4,201208,1000693,22954,공포,Invictus,5
...,...,...,...,...,...,...
9920,201208,1446850,769,공포,Goodfellas,6
9921,201208,1446850,6977,공포,No Country for Old Men,7
9922,201208,1446850,5915,공포,Into the Wild,8
9923,201208,1446850,8321,공포,In Bruges,9
